In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse.linalg import svds, eigs
from ast import literal_eval

%matplotlib inline

In [4]:
beer = pd.read_csv("../data400_share/beer.csv")
beer = beer[pd.notnull(beer['review/text'])]

In [34]:
count_vect = CountVectorizer(
    stop_words = 'english'
    #ngram_range = (1),
)

tfidf_transformer = TfidfTransformer()

In [35]:
## Vectorize the reviews
X_train_counts = count_vect.fit_transform(beer['review/text'])
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [36]:
u, s, vt = svds(X_train_tfidf, k = 100)

In [37]:
u.shape

(37490, 100)

In [38]:
s.shape

(100,)

In [39]:
vt.shape

(100, 41133)

In [5]:
## Organize Time structure
timeValues = ['min', 'hour', 'mday', 'sec', 'year', 'wday', 'mon', 'isdst', 'yday']

In [6]:
## All of the dictionaries of time for each review
dicts = [np.array(list(literal_eval(x).values())) for x in beer['review/timeStruct']]

In [7]:
## Create a Dataframe 
timesData = pd.DataFrame(dicts, columns=timeValues)

In [8]:
## Append to original beer DataFrame
beer_full = beer.join(timesData)

Append Kaggle beer Data

In [9]:
beer_full.groupby('year')['index'].count()

year
1999.0       4
2000.0       7
2001.0      27
2002.0     417
2003.0     861
2004.0    1175
2005.0    1969
2006.0    2931
2007.0    3035
2008.0    4496
2009.0    6536
2010.0    7303
2011.0    8422
2012.0     297
Name: index, dtype: int64

In [10]:
## 2009 year month weather data for breweries

In [11]:
bidname = pd.read_csv("beerDat.csv")
breweries = pd.read_csv("breweries.csv")

In [12]:
# Merge on id to get names
beerval = beer_full.merge(bidname, left_on="beer/brewerId", right_on="brewery_id")
# Get city, state by brewery name
beer_city = beerval.merge(breweries, left_on="brewery_name", right_on="name")

In [13]:
beer_city[['city', 'state']].drop_duplicates()

,city,state
0,Cambridge,MA
834,Grand Rapids,MI
15804,O'Fallon,MO
17081,Ashland,OR
17747,Charlotte,NC
17762,Morganton,NC
17892,Kirkwood,MO


In [14]:
beer_city.columns

Index(['index', 'beer/ABV', 'beer/beerId', 'beer/brewerId', 'beer/name',
       'beer/style', 'review/appearance', 'review/aroma', 'review/overall',
       'review/palate', 'review/taste', 'review/text', 'review/timeStruct',
       'review/timeUnix', 'user/ageInSeconds', 'user/birthdayRaw',
       'user/birthdayUnix', 'user/gender', 'user/profileName', 'min', 'hour',
       'mday', 'sec', 'year', 'wday', 'mon', 'isdst', 'yday', 'brewery_id',
       'brewery_name', 'Unnamed: 0', 'name', 'city', 'state'],
      dtype='object')

In [15]:
beer_city_sub = beer_full[[
    'user/profileName',
    'beer/ABV',
    'beer/brewerId',
    'beer/name',
    'review/appearance',
    'review/aroma',
    'review/overall',
    'review/palate',
    'review/taste',
    'review/text',
    'min',
    'hour',
    'mday',
    'sec',
    'year',
    'wday',
    'mon',
    'isdst',
    'yday'
]]

In [16]:
## Create all of the dummies
name_d = pd.get_dummies(beer_city_sub['beer/name'])
hour_d = pd.get_dummies(beer_city_sub['hour'], prefix = "hr_")
year_d = pd.get_dummies(beer_city_sub['year'], prefix = "yr_")
mon_d = pd.get_dummies(beer_city_sub['mon'], prefix = "mon_")
mday_d = pd.get_dummies(beer_city_sub['mday'], prefix = "mday_")
brewer_d = pd.get_dummies(beer_city_sub['beer/brewerId'], prefix= "brwid_")
wday_d = pd.get_dummies(beer_city_sub['wday'], prefix =  "wday_")

# Only select name with more than 5 values
beer_counts = beer.groupby(['beer/name'], as_index=False)[['index']].count()
beer_counts_s = beer_counts.sort_values(by = ['index'], ascending=False)
beer_filtered = beer_counts_s[beer_counts_s['index'] > 5]
new_cls = [col for col in name_d.columns if col in list(beer_filtered['beer/name'])]
name_d = name_d[new_cls]

In [17]:
beer_dummied = pd.concat([beer_city_sub, name_d, hour_d, year_d,mday_d, mon_d, brewer_d, wday_d], axis=1, sort=False)

In [18]:
beer_dummied.shape

(37490, 808)

In [19]:
beer_dropped = beer_dummied.drop([
    'beer/brewerId',
    'beer/name',
    'review/text',
    'hour',
    'year',
    'mon',
    'yday',
    'mday',
    'wday',
    'sec',
    'min'
], axis = 1)

In [20]:
new_dat =  pd.read_csv("fixed_beer_data.csv")

In [21]:
dat = new_dat.rename(
    index=str,
    columns = {
        'beer_name' : 'beer/name',
        'review_aroma' : 'review/aroma',
        'review_taste' : 'review/taste',
        'review_appearance' : 'review/appearance',
        'review_overall' : 'review/overall',
        'review_profilename' : 'user/profileName',
        'review_palate' : 'review/palate',
        'brewery_id' :  'beer/brewerId'
    }
)

In [22]:
dat = dat.drop(['beer_style', 'count'], axis = 1)

In [23]:
dat.shape

(37500, 13)

In [24]:
beer_full.shape

(37490, 28)

In [25]:
out = pd.merge(beer_dummied,
         dat,
         on = ['beer/name',
               'review/aroma',
               'review/taste',
               'review/appearance',
               'review/overall',
               'review/palate',
               'user/profileName',
               'beer/brewerId'],
         how = 'inner').dropna()







In [26]:
out.shape

(37489, 813)

In [30]:
out_dropped = out.drop([
    'user/profileName',
    'beer/brewerId',
    'beer/name',
    'hour',
    'year',
    'mon',
    'yday',
    'mday',
    'wday',
    'sec',
    'min'
], axis = 1)

In [31]:
out_dropped.shape

(37489, 802)

In [29]:
out_dropped.to_csv("full_data.csv")